In [63]:
import xml.etree.ElementTree as ET
import csv
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import RDF

def leggi_file_tsv(file_tsv):
    with open(file_tsv, 'r', encoding='utf-8') as tsvfile:
        reader = csv.DictReader(tsvfile, delimiter='\t')
        data = {row['DRUG_NAME'].lower(): row['MOA_SOURCE_URL'] if row['MOA_SOURCE_URL'] else row['ACT_SOURCE_URL'] if row['ACT_SOURCE_URL'] else 'null' for row in reader}
    return data

def genera_triple_rdf(xml_root, file_tsv):
    data_tsv = leggi_file_tsv(file_tsv)
    
    graph = Graph()
    rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
    ex = Namespace("http://example.org/")
    graph.bind("rdf", rdf)
    graph.bind("ex", ex)

    study = xml_root

    brief_title = study.find('./brief_title').text
    condition = study.find('./condition').text

    intervention_type = study.find('.//intervention_type').text
    intervention_name = study.find('.//intervention_name').text.lower()  # Converti in minuscolo

    if intervention_type.lower() == 'drug' and intervention_name in data_tsv:
        moa_source_url = data_tsv[intervention_name]

        # Aggiungi triple al grafo RDF
        subject = ex[intervention_name.replace(" ", "_")]  # Sostituisci spazi con underscore per creare URIs valide
        graph.add((subject, RDF.type, ex['Drug']))
        graph.add((subject, ex['condition'], Literal(condition)))
        graph.add((subject, ex['intervention_name'], Literal(intervention_name)))
        graph.add((subject, ex['MOA_SOURCE_URL'], URIRef(moa_source_url)))

    # Converte il grafo RDF in JSON-LD
    json_ld_data = graph.serialize(format='json-ld', indent=4)
    print(json_ld_data)

In [64]:
# Sostituisci con i tuoi percorsi di file
file_xml = '/home/deck/Desktop/UNI/Project/AllPublicXML/NCT0000xxxx/NCT00000102.xml'
file_tsv = '/home/deck/Desktop/UNI/Project/drug.target.interaction.tsv'

tree = ET.parse(file_xml)
root = tree.getroot()

genera_triple_rdf(root, file_tsv)


[
    {
        "@id": "http://example.org/nifedipine",
        "@type": [
            "http://example.org/Drug"
        ],
        "http://example.org/MOA_SOURCE_URL": [
            {
                "@id": "https://pubmed.ncbi.nlm.nih.gov/17276408"
            }
        ],
        "http://example.org/condition": [
            {
                "@value": "Congenital Adrenal Hyperplasia"
            }
        ],
        "http://example.org/intervention_name": [
            {
                "@value": "nifedipine"
            }
        ]
    }
]
